In [273]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
import numpy as np
%matplotlib inline
plt.style.use('Solarize_Light2')
import statistics
import talib as ta
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor as RF
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [200]:
df2 = pd.read_csv("Crypto data.csv") # Importing dataset

In [66]:
def ROC(df, n):  
    M = df.diff(n - 1)  
    N = df.shift(n - 1)  
    ROC = pd.Series(((M / N) * 100), name = 'ROC_' + str(n))   
    return ROC

In [69]:
def add_stochastic_oscillator(df, periods=14):
    copy = df.copy()
    
    high_roll = copy["High"].rolling(periods).max()
    low_roll = copy["Low"].rolling(periods).min()
    
    # Fast stochastic indicator
    num = copy["Close"] - low_roll
    denom = high_roll - low_roll
    copy["%K"] = (num / denom) * 100
    
    # Slow stochastic indicator
    copy["%D"] = copy["%K"].rolling(3).mean()
    
    return copy["%K"].values


In [ ]:
def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max() 
    lowl = low.rolling(lookback).min()
    wr = -100 * ((highh - close) / (highh - lowl))
    return wr

In [ ]:
def simple_exp_smooth(d,extra_periods=1,alpha=0.2):  
    d = np.array(d)  # Transform the input into a numpy array  
    cols = len(d)  # Historical period length  
    d = np.append(d,[np.nan]*extra_periods)  # Append np.nan into the demand array to cover future periods  
    f = np.full(cols+extra_periods,np.nan)  # Forecast array  
    f[1] = d[0]  # initialization of first forecast  
    # Create all the t+1 forecasts until end of historical period  
    for t in range(2,cols+1):  
        f[t] = alpha*d[t-1]+(1-alpha)*f[t-1]  
        f[cols+1:] = f[t]  # Forecast for all extra periods  
        df = pd.DataFrame.from_dict({"Simple Smoothing Data":f})
    return f

In [282]:
def creating_ds(df):
    if len(df) < 10:
        return None
    #df["SES_Open"] = simple_exp_smooth(df["Open"].values)
    #df["SES_Close"] = simple_exp_smooth(df["Close"].values)
    model = SimpleExpSmoothing(np.asarray(df['Open'])) #Smoothing the Open and Close values for further modelling
    #model._index = pd.to_datetime(train.index)
    model2 = SimpleExpSmoothing(np.asarray(df['Close']))
    #model._index = pd.to_datetime(train.index)

    OBV = []
    OBV.append(0)
    fit = model.fit(smoothing_level=.2)
    fit2 = model.fit(smoothing_level=.2)
    df["SES_Close"] = fit2.fittedvalues
    df["SES_Open"] = fit.fittedvalues
    try:
        df.ta.macd(close='SES_Close', fast=12, slow=26, signal=9, append=True)
    except:
        pass
    #Calculate the On Balance Volume
    for i in range(1, len(df.SES_Close)):
        if df["SES_Close"].iloc[[i]].values[0] > df["SES_Close"].iloc[[i-1]].values[0]: #If the closing price is above the prior close price 
            OBV.append(OBV[-1] + df["Volume"].iloc[[i]].values[0]) #then: Current OBV = Previous OBV + Current Volume
        elif df["Close"].iloc[[i]].values[0] < df["Close"].iloc[[i-1]].values[0]:
            OBV.append( OBV[-1] - df["Volume"].iloc[[i]].values[0])
        else:
            OBV.append(OBV[-1])
            
    #Store the OBV and OBV EMA into new columns
    df['OBV'] = OBV
    df['OBV_EMA'] = df['OBV'].ewm(com=20).mean()
    df['ROC'] = ROC(df['SES_Close'], 12)
    df["RSI"] = ta.RSI(df['SES_Close'], timeperiod=14)
    df["%K"] = add_stochastic_oscillator(df)
    df["wr_14"] = get_wr(df['High'], df['Low'], df['SES_Close'], 14)

    
    return df

In [286]:
df_combined = [] #This is the list of all datasets for all coins

for sy in sorted(np.unique(df2.Symbol)): # iterating through the coins to create the datasets to model from
    df_sy = creating_ds(df2[df2["Symbol"] == sy])
    df_combined.append(df_sy)

C:\Users\banat\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1409: RuntimeWarning:

divide by zero encountered in log

C:\Users\banat\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1415: RuntimeWarning:

divide by zero encountered in log

C:\Users\banat\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1409: RuntimeWarning:

divide by zero encountered in log

C:\Users\banat\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1415: RuntimeWarning:

divide by zero encountered in log

C:\Users\banat\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1409: RuntimeWarning:

divide by zero encountered in log

C:\Users\banat\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1415: RuntimeWarning:

divide by zero encountered in log

C:\Users\banat\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:1409: RuntimeWarning:

divide by zero encountered in log

C:\Users\banat\anaconda3\lib\site-

In [ ]:
symbol_ = []
mae_ = []
mape_ = []

for df in df_combined:# Use numpy to convert to arrays
    mae__ = []
    mape__ = []
    try:
        df.dropna(inplace=True)
    except:
        continue
    if len(df) == 0:
        continue
    # Labels are the values we want to predict
    labels = np.array(df['Close'])
    # Remove the labels from the features
    # axis 1 refers to the columns
    features= df.drop(["Date","Symbol","Open","High","Low","Close","Volume","Market Cap"], axis = 1)
    # Saving feature names for later use
    feature_list = list(features.columns)
    # Convert to numpy array
    features = np.array(features)
    # Using Skicit-learn to split data into training and testing sets
    dates_to_start = [0,30,60,90,120,150,180,210,240,270,300,330,360,390,420]
    dates_to_end = [-421,-391,-361,-331,-301,-271,-241,-211,-181,-151,-121,-91,-61,-31,-1]
    if len(df) > (dates_to_start[-1] * 2):
        for start_d, end_d in zip(dates_to_start, dates_to_end):
            train_features, test_features, train_labels, test_labels = train_test_split(features[start_d:end_d], labels[start_d:end_d], test_size = 0.20, random_state = 42)
            regr = RF(random_state=0, bootstrap=15)
            regr.fit(train_features,train_labels)
            # Use the forest's predict method on the test data
            predictions = regr.predict(test_features)
            # Calculate the absolute errors
            # Print out the mean absolute error (mae)
            #print(np.unique(df.Symbol.values))
            mae__.append(mean_absolute_error(test_labels, predictions))
            mape__.append(mean_absolute_percentage_error(test_labels, predictions))
            #print('Mean Absolute Error:', mean_absolute_error(test_labels, predictions), 'degrees.')
            #print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(test_labels, predictions))

    else:
        try:
            train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.20, random_state = 42)
            regr = RF(random_state=0, bootstrap=15)
            regr.fit(train_features,train_labels)
            predictions = regr.predict(test_features)
            #print(np.unique(df.Symbol.values))
            mae__.append(mean_absolute_error(test_labels, predictions))
            mape__.append(mean_absolute_percentage_error(test_labels, predictions))
            #print('Mean Absolute Error:', mean_absolute_error(test_labels, predictions), 'degrees.')
            #print('Mean Absolute Percentage Error:', mean_absolute_percentage_error(test_labels, predictions))
        except:
            continue

  
    mae_.append(statistics.mean(mae__))
    mape_.append(statistics.mean(mape__))
    symbol_.append(np.unique(df.Symbol.values)[0])
    
    continue

In [309]:
results_df = pd.DataFrame(np.column_stack([symbol_, mae_, mape_]), 
                               columns=['Symbol', 'MAE', 'MAPE'])

In [310]:
results_df

,Symbol,MAE,MAPE
0,AIDOC,0.0002358596253226785,0.04312103339309194
1,AIM,0.003016045909936906,0.027744413249020692
2,AION,0.017860537329529733,0.03836299175758053


In [10]:
Websites used: 

https://randerson112358.medium.com/stock-trading-strategy-using-on-balance-volume-obv-python-77a7c719cdac
https://www.alpharithms.com/calculate-macd-python-272222/
https://www.simplilearn.com/exponential-smoothing-for-time-series-forecasting-in-python-article
https://machinelearningmastery.com/exponential-smoothing-for-time-series-forecasting-in-python/
http://www.andrewshamlet.net/2017/07/07/python-tutorial-roc/
https://www.alpharithms.com/relative-strength-index-rsi-in-python-470209/
https://medium.com/wwblog/create-a-stochastic-oscillator-in-python-a7da42473677
https://medium.com/codex/algorithmic-trading-with-williams-r-in-python-5a8e0db9ff1f
https://towardsdatascience.com/diy-simple-exponential-smoothing-with-python-dbb570d30fb5


SyntaxError: invalid syntax (1267618460.py, line 1)